In [12]:
! apt install wkhtmltopdf
! pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 markdown pdfkit


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [13]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [14]:
from crewai import Agent, Task, Crew

In [15]:
import os
import getpass

# Prompt the user for each API key and save it into environment variables
serper_api_key = getpass.getpass("Enter SERPER_API_KEY: ")
openai_api_key = getpass.getpass("Enter OPENAI_API_KEY: ")

# Set the environment variables
os.environ["SERPER_API_KEY"] = serper_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"]="gpt-4o-mini"
# Confirm the values have been set (for debugging purposes)
print("Environment variables have been set.")


Environment variables have been set.


In [16]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./resume.md')
semantic_search_resume = MDXSearchTool(mdx='./resume.md')

In [17]:
# Agent 1: Job Description Scraper and Analyzer
job_analyzer = Agent(
    role="Tech Job Researcher and Analyzer",
    goal="Scrape the job posting URL, extract the job description, and analyze key requirements.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

# Agent 2: Relevance Selector
relevance_selector = Agent(
    role="Relevance Selector",
    goal="Identify and justify the most relevant sections in the resume based on the job description analysis.",
    tools=[read_resume, semantic_search_resume, search_tool],
    verbose=True,
    backstory="You excel at matching resume content to job requirements, using semantic search to find the most relevant information."
)

# Agent 3: Emphasis Strategist
emphasis_strategist = Agent(
    role="Emphasis Strategist",
    goal="Strategically suggest words or phrases to bold, explaining their significance in relation to the job requirements.",
    tools=[read_resume, semantic_search_resume, search_tool],
    verbose=True,
    backstory="Your expertise lies in highlighting key terms that will catch a recruiter's eye and demonstrate the candidate's fit for the role."
)

# Agent 4: Cover Letter Writer
cover_letter_writer = Agent(
    role="Cover Letter Writer",
    goal="Craft a compelling, tailored cover letter that highlights the candidate's qualifications and enthusiasm for the role.",
    tools=[read_resume, semantic_search_resume, search_tool],
    verbose=True,
    backstory="You are a master of persuasive writing, able to create cover letters that effectively showcase a candidate's fit for a specific role and company culture."
)



In [18]:
# Task 1: Scrape and Analyze Job Description
job_analysis_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. if you cannot scrape the job description or no relevant informationis available from the URL use ({job_description}) which is an alternative but unstructured version of the job posting, Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=job_analyzer
)

# Task 2: Identify Relevant Sections
relevance_task = Task(
    description=(
        "Using the semantic_search_resume tool, identify the most relevant sections of the super resume that align "
        "with the job analysis. Provide a thorough justification for each selected section, explaining how it "
        "demonstrates the candidate's fit for the role. Use the semantic search to find not just keyword matches, "
        "but conceptually relevant experiences and skills."
    ),
    expected_output=(
        "A shortened resume tailored to the job requirements. Stay true to the orignal resume and do not hallucinate. Follow the existing resume format and keep the following on the focused resume:\n"
        "1. the list of skills focused towards the job description but a few more from the existing list to show the breadth of skills by following the section's original format\n"
        "2. A minimum of 4 jobs under work experience and a total of 10 bullt points\n"
        "4. A minimum of 2 and maximum of 3 projects with the same bullet points with possible minor improvements.\n"
        "5. Make changes to potential areas where the section could be enhanced to better match the job description\n"
        "6. Bold all keywords which will helpa recruiter quicly analyze the created resume\n"
        "7. A brief explanation of key decisions made in crafting the resume, including:\n"
        "   - Choice of highlighted experiences and skills\n"
        "   - How the resume addresses specific job requirements\n"
        "   - Any company-specific references or customizations"
        "   - Any other customizations"

    ),
    output_file="focused_resume.md",
    agent=relevance_selector,
    context=[job_analysis_task]
)

# Task 3: Suggest Words/Phrases to Bold
emphasis_task = Task(
    description=(
        "Analyze the selected relevant sections and the job requirements to identify key words or phrases that should "
        "be bolded to enhance the resume's impact. Use the semantic_search_resume tool to identify terms that are "
        "conceptually important, not just exact matches. For each suggestion, provide a clear rationale explaining its "
        "significance in relation to the job requirements and how it will catch the attention of recruiters or ATS systems."
    ),
    expected_output=(
        "A comprehensive list of words/phrases to be bolded, organized by resume section. For each suggestion, include:\n"
        "1. The word or phrase to be bolded\n"
        "2. The resume section it appears in\n"
        "3. A detailed explanation of why this term is crucial, referencing specific job requirements\n"
        "5. If applicable, suggest minor rewording to better align the term with the job description language\n"
        "6. The semantic search query used to identify this term (if applicable)"
        "7. Strategically bold phrases/words to avoid the recruiter only reading in the F shaped patter and end up avoiding any critical information"
    ),
    output_file="emphasize_strategy.md",
    agent=emphasis_strategist,
    context=[job_analysis_task, relevance_task]
)

# Task 3: Generate Cover Letter
cover_letter_task = Task(
    description=(
        "Using the job analysis and resume optimization insights, craft a compelling cover letter that highlights "
        "the candidate's qualifications and enthusiasm for the role. Tailor the letter to the specific job and company."
    ),
    expected_output=(
        "A well-structured, tailored cover letter that includes:\n"
        "1. An engaging opening paragraph that captures attention and states the purpose\n"
        "2. 2-3 body paragraphs that highlight the candidate's most relevant qualifications, using specific examples from the resume\n"
        "3. A closing paragraph that reiterates interest and suggests next steps\n"
        "4. Appropriate tone and style for the industry and company culture\n"
        "5. A brief explanation of key decisions made in crafting the letter, including:\n"
        "   - Choice of highlighted experiences and skills\n"
        "   - How the letter addresses specific job requirements\n"
        "   - Any company-specific references or customizations"
    ),
    output_file="cover_letter.md",
    agent=cover_letter_writer,
    context=[job_analysis_task, relevance_task]
)

In [19]:

# Crew setup
job_application_crew = Crew(
    agents=[job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer],
    tasks=[job_analysis_task, relevance_task, emphasis_task,  cover_letter_task],
    verbose=True
)

# Inputs for the job application process
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/4011986711/?alternateChannel=search&refId=p02ZIOilkaTOINMOOZOPEA%3D%3D&trackingId=xnldLTQp8%2BpuVplwxN6OkQ%3D%3D',
    'super_resume_path': './super_resume.md',
    'job_description': '''
             Stealth
AI Research Scientist [28557]
Cupertino, CA 


About:

We are a Series A startup focused on developing cutting-edge compressed models for devices. Our mission is to bring powerful AI capabilities to edge devices, enabling seamless integration of advanced language models into everyday technology. Join us as we push the boundaries of AI, creating impactful solutions optimized for real-world applications.


Responsibilities:

    Create datasets for potential, powerful capabilities of language models such as function calling and reflection
    Build tooling and infrastructure to enable efficient fine-tuning experiments on language models
    Help develop new methods or novel fine-tuning techniques to improve language model behaviors
    Run experiments that feed into key AI research


You may be a good fit if you:

    Have at least 1 research project related to machine learning in which you played a major role
    Significant Python, machine learning, research experience
    Are results-oriented, with a bias towards urgency, impact, and flexibility
    Be familiar with C or C++


        '''
}

# Execute the job application process
result = job_application_crew.kickoff(inputs=job_application_inputs)

print("\nJob Application Package:")
print(result)

# # Save the result to a file
# with open('cover_letter.md', 'w') as f:
#     f.write(result)
# print("\nJob application package saved to 'job_application_package.txt'")

# with open('cover_letter.md', 'w') as f:
#     f.write(resume_optimization_task.output.exported_output)
# print("\nJob application package saved to 'job_application_package.txt'")


2024-09-01 11:35:32,827 - 140374349153344 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Tech Job Researcher and Analyzer
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.linkedin.com/jobs/view/4011986711/?alternateChannel=search&refId=p02ZIOilkaTOINMOOZOPEA%3D%3D&trackingId=xnldLTQp8%2BpuVplwxN6OkQ%3D%3D) to extract key skills, experiences, and qualifications required. if you cannot scrape the job description or no relevant informationis available from the URL use (
             Stealth
AI Research Scientist [28557]
Cupertino, CA 


About:

We are a Series A startup focused on developing cutting-edge compressed models for devices. Our mission is to bring powerful AI capabilities to edge devices, enabling seamless integration of advanced language models into everyday technology. Join us as we push the boundaries of AI, creating impactful solutions optimized for real-world applications.


Responsibilities:

    Create datasets for potential, powerful capabilities of language models such as function calling and reflecti

2024-09-01 11:35:33,205 - 140372894238272 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.6s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fab105a1390>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-09-01 11:35:33,828 - 140372894238272 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.8s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fab105b1510>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-09-01 11:35:34,711 - 140372894238272 - _common.py-_common:105 - INFO: Backing off send_request(...) for 2.7s (req

I need to extract the job description and analyze key requirements from the provided LinkedIn job posting URL. I will start by attempting to read the website content at the given URL.

Action: Read website content
Action Input: {"website_url": "https://www.linkedin.com/jobs/view/4011986711/?alternateChannel=search&refId=p02ZIOilkaTOINMOOZOPEA%3D%3D&trackingId=xnldLTQp8%2BpuVplwxN6OkQ%3D%3D"} 





2024-09-01 11:35:37,586 - 140372894238272 - _common.py-_common:120 - ERROR: Giving up send_request(...) after 4 tries (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fab003f72e0>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))


Thought: The attempt to read the website content did not yield any results. Therefore, I will use the alternative unstructured job posting information provided to extract and categorize the key skills, qualifications, and experiences required for the AI Research Scientist position.

Final Answer:
```plaintext
Job Title: AI Research Scientist

Location: Cupertino, CA

About:
We are a Series A startup focused on developing cutting-edge compressed models for devices. Our mission is to bring powerful AI capabilities to edge devices, enabling seamless integration of advanced language models into everyday technology. Join us as we push the boundaries of AI, creating impactful solutions optimized for real-world applications.

Responsibilities:
- Create datasets for potential, powerful capabilities of language models such as function calling and reflection
- Build tooling and infrastructure to enable efficient fine-tuning experiments on language models
- Help develop new methods or novel fine-

In [20]:
import markdown
import pdfkit
import os

def convert_md_to_pdf(md_files):
    for md_file in md_files:
        # Check if the file exists
        if not os.path.exists(md_file):
            print(f"The file '{md_file}' does not exist. Skipping...")
            continue
        
        # Generate the PDF file name by replacing the .md extension with .pdf
        pdf_file = os.path.splitext(md_file)[0] + '.pdf'
        
        # Read the Markdown file
        with open(md_file, 'r', encoding='utf-8') as f:
            md_content = f.read()

        # Convert Markdown to HTML
        html_content = markdown.markdown(md_content)

        # Convert HTML to PDF
        pdfkit.from_string(html_content, pdf_file)

        print(f"Markdown file '{md_file}' has been converted to '{pdf_file}'.")

convert_md_to_pdf(["focused_resume.md", "cover_letter.md", "emphasize_strategy.md"])

Markdown file 'focused_resume.md' has been converted to 'focused_resume.pdf'.
Markdown file 'cover_letter.md' has been converted to 'cover_letter.pdf'.
Markdown file 'emphasize_strategy.md' has been converted to 'emphasize_strategy.pdf'.


In [21]:
from IPython.display import Markdown, display
display(Markdown("./job_application_package.txt"))

[Your Name]  
[Your Address]  
[City, State, Zip]  
[Your Email]  
[Your Phone Number]  
[Date]  

Hiring Manager  
Walmart Global Tech  
[Company Address]  
[City, State, Zip]  

Dear Hiring Manager,

I am writing to express my enthusiasm for the Senior Data Scientist - Computer Vision Engineer position at Walmart, as advertised. With a Master’s degree in Robotics and Autonomous Systems, coupled with over five years of hands-on experience in data science and machine learning, particularly in computer vision, I am excited about the opportunity to contribute to Walmart's mission of leveraging AI technologies to enhance operations and customer experiences.

At RagaAI, I spearheaded the deployment of a deep neural network for drift tracking and outlier detection in Advanced Driver-Assistance Systems (ADAS), achieving a remarkable 95% test accuracy. This project not only showcased my technical proficiency in machine learning frameworks such as TensorFlow and PyTorch but also my ability to deliver impactful solutions in high-stakes environments. My collaboration on developing the RagaAI Platform for computer vision applications allowed me to refine my skills in model deployment and anomaly detection, which directly contributed to securing $4.7 million in seed funding. These experiences have honed my analytical and problem-solving skills, essential for a fast-paced environment like Walmart.

Throughout my career, I have consistently prioritized the integration of cutting-edge technologies with practical applications. My recent project involved creating a stress-testing framework for object detectors, which simulated real-life scenarios to test model robustness. This aligns perfectly with Walmart's commitment to innovation and operational excellence. Additionally, my proficiency in cloud computing platforms like AWS and GCP ensures that I can effectively deploy and manage machine learning models in production settings, further enhancing Walmart’s technological capabilities.

I am particularly drawn to Walmart's vision of utilizing AI to better serve millions of customers worldwide. I am eager to bring my expertise in computer vision and collaborative spirit to the Global Tech team, contributing to projects that drive tangible improvements in customer experience and operational efficiencies. I am committed to staying at the forefront of industry trends and advancements, ensuring that my contributions are not only relevant but also transformative.

Thank you for considering my application. I look forward to the opportunity to discuss how my background, skills, and enthusiasms align with the goals of Walmart. I am excited about the possibility of contributing to such an impactful organization and am eager to bring my expertise in computer vision to help drive innovation at Walmart.

Warm regards,

[Your Name]

---

### Explanation of Key Decisions Made in Crafting the Letter:
1. **Engaging Opening:** The opening paragraph establishes the candidate's enthusiasm for the role and briefly mentions their relevant educational background and experience.
2. **Highlighting Relevant Qualifications:** The body paragraphs focus on specific experiences that align with the job requirements, such as deploying machine learning models, achieving high accuracy in projects, and collaborating on successful initiatives.
3. **Company Alignment:** The letter emphasizes the candidate's desire to contribute to Walmart's mission, showcasing understanding and alignment with the company’s goals.
4. **Professional Tone:** The language used is formal yet engaging, reflecting a professional demeanor while expressing genuine enthusiasm for the role. 

This structured approach ensures that the cover letter effectively showcases the candidate's qualifications while demonstrating their excitement for the opportunity at Walmart.